<a href="https://colab.research.google.com/github/amriT2044524/geemap_amrit/blob/main/spectral_profile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import geemap

ee.Authenticate() # https://book.geemap.org/chapters/01_introduction.html#earth-engine-authentication
ee.Initialize(project='ee-athapa2')


In [ ]:
# 1. Define the point of interest (optional)
my_lon = 81.683341
my_lat = 30.205756
point = ee.Geometry.Point(my_lon, my_lat)

# 2. Filter Sentinel-2 and get the least-cloudy image
s2_collection = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(point)
    .filterDate('2024-01-01', '2024-12-31')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
    .sort('CLOUDY_PIXEL_PERCENTAGE')
)

least_cloud_image = s2_collection.first()

# 3. Select bands for plotting
bands = ['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']
s2_image = least_cloud_image.select(bands)

# 4. Create Map
m = geemap.Map(center=[my_lat, my_lon], zoom=15)

# Add image layer
vis_params = {'min': 0, 'max': 3000, 'bands': ['B8', 'B4', 'B3']}
m.add_layer(s2_image, vis_params, 'Least Cloudy S2')

# Add marker for your original point (optional)
m.add_marker(location=[my_lat, my_lon], tooltip="Original Point")

# 5. Add the interactive plot GUI
m.add_plot_gui()

# Display
m
